## Data Transformation

In [2]:
import os
%pwd

'c:\\Users\\dipgh\\OneDrive\\Desktop\\MLOps\\TextSummarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\dipgh\\OneDrive\\Desktop\\MLOps\\TextSummarizer'

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_inp_path: Path
    data_path: Path
    tokenizer_name: Path


In [29]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_inp_path=config.data_inp_path,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [35]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk, DatasetDict

In [40]:
# class DataTransformation:
#     def __init__(self,config:DataTransformationConfig):
#         self.config=config
#         self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

#     def convert_examples_to_features(self,example_batch):
#         input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

#         with self.tokenizer.as_target_tokenizer():
#             target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

#         return {
#             'input_ids' : input_encodings['input_ids'],
#             'attention_mask': input_encodings['attention_mask'],
#             'labels': target_encodings['input_ids']
#         }
    
#     def convert(self):
#         dataset_samsum = load_from_disk(self.config.data_path)
#         dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
#         dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        # This method remains the same as it handles the tokenization
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)
            
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        # Load the pre-split datasets from json files
        train_dataset = load_dataset("json", 
                                   data_files=os.path.join(self.config.data_inp_path, "train.json"))["train"]
        test_dataset = load_dataset("json", 
                                  data_files=os.path.join(self.config.data_inp_path, "test.json"))["train"]
        val_dataset = load_dataset("json", 
                                 data_files=os.path.join(self.config.data_inp_path, "val.json"))["train"]
        
        # Create a DatasetDict with all splits
        dataset_samsum = DatasetDict({
            "train": train_dataset,
            "test": test_dataset,
            "validation": val_dataset
        })

        # Apply tokenization to all splits
        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features, 
            batched=True,
            remove_columns=dataset_samsum["train"].column_names  # Remove original columns after tokenization
        )
        
        # Save the processed dataset
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.data_path))

In [41]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2025-04-21 13:28:24,934]: INFO: common: yaml file: config\config.yaml loaded successfully:
[2025-04-21 13:28:24,935]: INFO: common: yaml file: params.yaml loaded successfully:
[2025-04-21 13:28:24,936]: INFO: common: created directory at: artifacts:
[2025-04-21 13:28:24,937]: INFO: common: created directory at: artifacts/:


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 138803.33 examples/s]
